## Data Cleaning

This script cleans the raw annotated data, counts number of annotators, and outputs processed annotation data to be merged with CANDOR transcripts.

**Authors:** Changyi Zhou, Helen Schmidt  
**Python version:** 3.11.13

In [1]:
import pandas as pd
import json
import numpy as np
from IPython.display import display
import os

In [2]:
# define data input location
input_folder = "/Users/helenschmidt/Library/CloudStorage/GoogleDrive-helenschmidt129@gmail.com/My Drive/SANLab/Experiments/Conversation-Structure/data/raw/full-sample"
# define data output location
output_folder = "/Users/helenschmidt/Library/CloudStorage/GoogleDrive-helenschmidt129@gmail.com/My Drive/SANLab/Experiments/Conversation-Structure/data/processed/full-sample"

In [3]:
# get files in the input folder from the full sample
files = [file for file in os.listdir(input_folder) if file.endswith('.csv')]
# check the raw files look correct
print(files)

['full_data_pilot_4:18.csv', 'full_data_s2.csv', 'full_data_s2-5.csv', 'full_data_s2-4.csv', 'full_data_s2-3.csv', 'full_data_s2-2.csv', 'lowqual2.csv', 'lowqual3.csv', 'lowqual1.csv', 'full_data_s1-1.csv', 'full_data_s1-2.csv', 'full_data_s1-3.csv']


In [4]:
# create empty data frame
dataframes = []
# count the values and occurrences of annotations in each data file
for file in files:
    file_path = os.path.join(input_folder, file)
    df = pd.read_csv(file_path)
    if 'messages' in df.columns:
        messages_df = df[df['messages'].notna() & (df['messages'].str.strip() != '')]
        if 'data' in messages_df.columns:
            messages_df = messages_df.drop(columns=['data'])

        dataframes.append(messages_df)

        if 'count' in messages_df.columns:
            count_values = messages_df['count'].value_counts()

            count_values_df = pd.DataFrame({
                'count_value': count_values.index,
                'occurrences': count_values.values
            })

            print (f'{file} - Total submissions: {count_values_df["occurrences"].sum()}')

        else:
            print("The 'count' column was not found in the messages DataFrame.")

raw_combined = pd.concat(dataframes, ignore_index=True)

# print final number
print (f'Total submissions across all data files: {len(raw_combined)}')
# display(raw_combined)

full_data_pilot_4:18.csv - Total submissions: 144
full_data_s2.csv - Total submissions: 285
full_data_s2-5.csv - Total submissions: 54
full_data_s2-4.csv - Total submissions: 123
full_data_s2-3.csv - Total submissions: 122
full_data_s2-2.csv - Total submissions: 203
lowqual2.csv - Total submissions: 26
lowqual3.csv - Total submissions: 21
lowqual1.csv - Total submissions: 92
full_data_s1-1.csv - Total submissions: 208
full_data_s1-2.csv - Total submissions: 259
full_data_s1-3.csv - Total submissions: 135
Total submissions across all data files: 1672


In [5]:
# get dense subset csv
dense_subset = pd.read_csv('/Users/helenschmidt/Library/CloudStorage/GoogleDrive-helenschmidt129@gmail.com/My Drive/SANLab/Experiments/Conversation-Structure/data/raw/full_dense_subset_raw.csv')
# get list of transcript ids
dense_transcripts = dense_subset['transcript_id'].unique()
print (f'Total annotated transcripts in dense subset: {len(dense_transcripts)}')

Total annotated transcripts in dense subset: 200


In [6]:
# check that remaining conversations from the CANDOR data set have been annotated by at least one person (N = 1456 unique transcripts; doesn't include dense subset of 200)

expected_counts = set(range(1456))  # 0 to 1455 inclusive
all_counts = set()

for file in os.listdir(input_folder):
    if not file.endswith('.csv'):
        continue

    file_path = os.path.join(input_folder, file)

    try:
        df = pd.read_csv(file_path, usecols=['messages', 'count'])  # only load needed columns
    except ValueError:
        print(f"File {file} is missing required columns.")
        continue

    # Filter rows with valid messages
    filtered = df['messages'].fillna('').str.strip() != ''
    counts = df.loc[filtered, 'count'].dropna().astype(int).unique()

    all_counts.update(counts)

missing_counts = expected_counts - all_counts

print(f"Missing count values: {sorted(missing_counts)}")
print(len(missing_counts))

Missing count values: []
0


In [7]:
# save processed data file
final_df = pd.DataFrame()
count = 0

for file in files:
    file_path = os.path.join(input_folder, file)
    df = pd.read_csv(file_path)
    for idx, msg in df['messages'].items():
        if pd.isna(msg):
            continue

        try:
            data = json.loads(msg)
        except json.JSONDecodeError:
            print(f"Skipping row {idx} due to JSON decoding error.")
            continue
        data_df = pd.DataFrame(data)

        data_df['transcript_id'] = df.loc[idx, 'conversation_id']

        data_df['count'] = df.loc[idx, 'count']

        final_df = pd.concat([final_df, data_df], ignore_index=True)

        count += 1

# fix spelling on participant ID
final_df.rename(columns={'participent_id': 'PID'}, inplace=True)

# save data file to processed folder
save_path = output_folder + "/" + "full_sample_processed.csv"
final_df.to_csv(save_path, index=False)

print(f"Total valid rows processed: {count}")

Total valid rows processed: 1672


In [7]:
# print number of unique transcripts
print(f"Number of unique annotated transcripts = {final_df['transcript_id'].nunique()}")

# make PID a string instead of a list
final_df['PID_str'] = final_df['PID'].apply(lambda x: ''.join(map(str, x)))

# get count of annotations per PID
annotation_count = final_df.groupby('PID_str')['new_topic'].nunique().reset_index()
annotation_count.rename(columns = {'new_topic': 'PID_count'}, inplace=True)

# print average number of annotations per PID
print(f"Mean annotation count per participant = {annotation_count['PID_count'].mean()}")

# print the number of participants with fewer than 6 annotations
print(f"Number of participants with fewer than 6 annotations = {(annotation_count['PID_count'] < 6).sum()}")

Number of unique annotated transcripts = 1464
Mean annotation count per participant = 20.672738312082576
Number of participants with fewer than 6 annotations = 153


In [34]:
# define directory for CANDOR transcripts
folder_dir = '/Users/helenschmidt/Library/CloudStorage/GoogleDrive-helenschmidt129@gmail.com/My Drive/SANLab/Experiments/CANDOR/transcripts/raw'
# load all modified transcripts and combine into one data frame
all_transcripts = []
for dirpath, dirnames, filenames in os.walk(folder_dir):
    for filename in filenames:
        if filename == 'transcript_backbiter_transformed_noLine1.csv':
            file_path = os.path.join(dirpath, filename)
            df = pd.read_csv(file_path)
            relative_path = os.path.relpath(dirpath, folder_dir)
            transcript_id = relative_path.split(os.sep)[0] if relative_path else ''
            # add new variable for transcript ID from folder name
            df['transcript_id'] = transcript_id
            all_transcripts.append(df)

# Concatenate all dataframes by rows (like row bind)
all_transcripts = pd.concat(all_transcripts, ignore_index=True)

# get list of transcript names in the directory
folder_names = [name for name in os.listdir(folder_dir) if os.path.isdir(os.path.join(folder_dir, name))]

# now get list of transcript IDs from 
transcript_ids = final_df['transcript_id'].unique()
# concatenate dense subset IDs with full sample IDs
combined_transcripts = np.concatenate((transcript_ids, dense_transcripts))

# convert lists to sets to efficiently compare
folder_set = set(folder_names)
id_set = set(combined_transcripts)

# find transcripts in CANDOR data that are not in the list of IDs in the list
unannotated_transcripts = folder_set - id_set

# print missing transcripts
print(f"Transcripts not in annotation list ({len(unannotated_transcripts)}):")
for folder in sorted(unannotated_transcripts):
    print(folder)

Transcripts not in annotation list (4):
43540c29-7c51-4fc8-8bda-fe8614340c64
765f6cde-5291-4047-89c1-d71b1e3a413d
7a47e9ed-6bfa-499f-811b-f9e2bffc2770
bfa73b4c-5b45-4c17-9fe4-8b4f397654e6
